# Booking.com Hotel Review NLP

Features by relationship

Hotel
- address
- hotel name
- lat
- long
- total_number_of_reviews
- additional_number_of_scoring
- average_score

reviewer
- review_nationality
- total_number_of_reviews_reviewer_has_given
- reviewer_score
- tags

review
- negative_review
- review_total_negative_word_counts
- negative_unique_word_count
- positive_review
- review_total_postive_word_counts
- positive_uniqu_word_count
- days_since_review

date
- month
- year
- day_name
- day
- quarter

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import re
import wrangle

pd.options.display.max_colwidth = None
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Acquire

- How do we aquire this from the webpage programatically?

In [2]:
hotel = pd.read_csv('Hotel_Reviews.csv')

In [3]:
hotel = wrangle.wrangle_hotel(hotel)

In [4]:
hotel.head(1)

,hotel_address,additional_number_of_scoring,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,month,year,day_name,day,quarter,negative_unique_word_count,positive_unique_word_count,location
review_date,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-04,225 Edgware Road Westminster Borough London W2 1JU United Kingdom,1485,7.5,Hilton London Metropole,United Kingdom,First day the swimming pool was not opened therefore i could not access the facilities Finaly the receptionist fid not clarify that i would unable to move my car and did not stamp my car parking so i can come in and out Therefore i had to pay 50 for one day,54,6977,The hotel was clean and accessible,8,4,7.5,"[leisure trip, group, superior queen room, stayed 2 nights, submitted from a mobile device]",730,51.519569,-0.170521,August,2015,Tuesday,4,3,39,6,United Kingdom


# Wrangle

### Wrangle Function

In [5]:
# def wrangle_hotel(df):
#     '''
#     Wrangle Start
#     '''
    
#     # lower case column names
#     df.columns = [col.lower() for col in df]
    
#     # Set the review date as a datetime object then set it as the index
#     df.review_date = pd.to_datetime(df.review_date)
#     df = df.set_index('review_date').sort_index()
    
#     # Create columns for date types to groupby
#     df['month'] = df.index.month_name()
#     df['year'] = df.index.year
#     df['day_name'] = df.index.day_name()
#     df['day'] = df.index.day
#     df['quarter'] = df.index.quarter
    
#     # Unique word counts for positive and negative reviews
#     df['negative_unique_word_count'] = [len(set(nr.split())) for nr in df.negative_review]
#     df['positive_unique_word_count'] = [len(set(pr.split())) for pr in df.positive_review]
    
#     # remove day string and make int type
#     df.days_since_review = [row.split()[0] for row in df.days_since_review]
#     df.days_since_review = df.days_since_review.astype('int')
    
#     # Get Hotel Location
#     df['location'] = [' '.join(col.split()[-2:]) for col in df.hotel_address]
    
#     # Break out tags into groups
#     df.tags = [[tag.strip().lower() for tag in (tags.replace('"','').replace("'","")
#                                      .replace('[','') .replace(']','')
#                                      .split(','))] for tags in df.tags]
    
#     return df

In [6]:
# lower case column names
# hotel.columns = [col.lower() for col in hotel]

In [7]:
# # Set the review date as a datetime object then set it as the index
# hotel.review_date = pd.to_datetime(hotel.review_date)
# hotel = hotel.set_index('review_date').sort_index()
# # Create columns for date types to groupby
# hotel['month'] = hotel.index.month_name()
# hotel['year'] = hotel.index.year
# hotel['day_name'] = hotel.index.day_name()
# hotel['day'] = hotel.index.day
# hotel['quarter'] = hotel.index.quarter

In [8]:
# # Unique word count for first row
# #len(set(hotel.negative_review[0].split()))

# # Unique word counts for positive and negative reviews
# hotel['negative_unique_word_count'] = [len(set(nr.split())) for nr in hotel.negative_review]
# hotel['positive_unique_word_count'] = [len(set(pr.split())) for pr in hotel.positive_review]

In [9]:
# # remove day string and make int type
# hotel.days_since_review = [row.split()[0] for row in hotel.days_since_review]
# hotel.days_since_review = hotel.days_since_review.astype('int')

In [10]:
# # Get Hotel Location
# hotel['location'] = [' '.join(col.split()[-2:]) for col in hotel.hotel_address]

In [11]:
# # Break out tags into groups
# hotel.tags = [[tag.strip().lower() for tag in (tags.replace('"','').replace("'","")
#                                      .replace('[','') .replace(']','')
#                                      .split(','))] for tags in hotel.tags]

### Address

In [12]:
# We need to figure out a way to seperate out the address in to its parts.
# Street Address
# City
# Zip
# Country
# hotel.hotel_address.value_counts()

In [13]:
hotel.hotel_address[0].split()[-2:]

['United', 'Kingdom']

In [14]:
hotel.location.value_counts()

United Kingdom           262301
Barcelona Spain           60149
Paris France              59928
Amsterdam Netherlands     57214
Vienna Austria            38939
Milan Italy               37207
Name: location, dtype: int64

### Review Lengths

In [15]:
# Length of negative review
len(hotel.negative_review[0].split())

52

In [16]:
# Given length of negative review seems wrong and should be investigated further
hotel.review_total_negative_word_counts[0]

54

### Tags

In [18]:
# pd.Series([len(tags.split(',')) for tags in hotel.tags]).value_counts().sort_index()

In [19]:
hotel.tags[0][0]

'leisure trip'

In [ ]:
# [tag for tag in hotel.tags if len(tag) == 6]
# [tag for tag in hotel.tags if len(tag) == 5]
# [tag for tag in hotel.tags if len(tag) == 4]
# [tag for tag in hotel.tags if len(tag) == 3]
# [tag for tag in hotel.tags if len(tag) == 2]
# [tag for tag in hotel.tags if len(tag) == 1]

In [20]:
hotel.head(1)

,hotel_address,additional_number_of_scoring,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,month,year,day_name,day,quarter,negative_unique_word_count,positive_unique_word_count,location
review_date,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-04,225 Edgware Road Westminster Borough London W2 1JU United Kingdom,1485,7.5,Hilton London Metropole,United Kingdom,First day the swimming pool was not opened therefore i could not access the facilities Finaly the receptionist fid not clarify that i would unable to move my car and did not stamp my car parking so i can come in and out Therefore i had to pay 50 for one day,54,6977,The hotel was clean and accessible,8,4,7.5,"[leisure trip, group, superior queen room, stayed 2 nights, submitted from a mobile device]",730,51.519569,-0.170521,August,2015,Tuesday,4,3,39,6,United Kingdom


# Explore

In [21]:
hotel.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,days_since_review,lat,lng,year,day,quarter,negative_unique_word_count,positive_unique_word_count
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,512470.000000,512470.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000
mean,498.081836,8.397487,18.539450,2743.743944,17.776458,7.166001,8.395077,354.441932,49.442439,2.823803,2016.120761,15.680221,2.503409,14.814799,14.307257
std,500.538467,0.548048,29.690831,2317.464868,21.804185,11.040228,1.637856,208.928124,3.466325,4.579425,0.687567,8.864491,1.088472,19.805462,15.847826
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,2.500000,0.000000,41.328376,-0.369758,2015.000000,1.000000,1.000000,0.000000,0.000000
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,7.500000,175.000000,48.214662,-0.143372,2016.000000,8.000000,2.000000,2.000000,4.000000
50%,341.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,8.800000,353.000000,51.499981,0.010607,2016.000000,16.000000,3.000000,7.000000,9.000000
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.000000,9.600000,527.000000,51.516288,4.834443,2017.000000,23.000000,3.000000,20.000000,19.000000
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,10.000000,730.000000,52.400181,16.429233,2017.000000,31.000000,4.000000,240.000000,234.000000


In [22]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 515738 entries, 2015-08-04 to 2017-08-03
Data columns (total 24 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   average_score                               515738 non-null  float64
 3   hotel_name                                  515738 non-null  object 
 4   reviewer_nationality                        515738 non-null  object 
 5   negative_review                             515738 non-null  object 
 6   review_total_negative_word_counts           515738 non-null  int64  
 7   total_number_of_reviews                     515738 non-null  int64  
 8   positive_review                             515738 non-null  object 
 9   review_total_positive_word_counts           515738 non

## Univariate

### Mising Values

In [23]:
# print nulls per row
[print(f'{col}:{hotel[col].isnull().sum()}') for col in  hotel.columns] # How to format better

hotel_address:0
additional_number_of_scoring:0
average_score:0
hotel_name:0
reviewer_nationality:0
negative_review:0
review_total_negative_word_counts:0
total_number_of_reviews:0
positive_review:0
review_total_positive_word_counts:0
total_number_of_reviews_reviewer_has_given:0
reviewer_score:0
tags:0
days_since_review:0
lat:3268
lng:3268
month:0
year:0
day_name:0
day:0
quarter:0
negative_unique_word_count:0
positive_unique_word_count:0
location:0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### No valid review

In [24]:
hotel.negative_review[2]

' Nothing'

In [25]:
(hotel.positive_review == 'No Positive').mean()

0.0696981800836859

In [26]:
(hotel.positive_review == ' Nothing').mean()

0.002410138481166794

In [27]:
(hotel.negative_review == 'No Negative').mean()

0.24797474686759557

In [28]:
(hotel.negative_review == ' Nothing').mean()

0.02771756201792383

In [29]:
# Unique Values per column
# for col in hotel.columns:
#     print(f'{col} Counts:{hotel[col].nunique()}')